In [23]:
import numpy as np
import pandas as pd
import random
import keras
import os
import math

from keras.models import Sequential
from keras.layers import Dense, Flatten, Reshape, Dropout
from keras.layers.convolutional import Conv2D, MaxPooling2D,ZeroPadding2D
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import concatenate
from keras.utils import plot_model

import scipy.io
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import matplotlib.gridspec as gridspec
from datetime import datetime

from keras.layers import Conv2D, LeakyReLU
from keras.callbacks import EarlyStopping
from keras import regularizers

from keras.optimizers import SGD
import cv2, numpy as np

In [16]:
src = 'data/'
save = 'save/'
files = os.listdir(src)


In [15]:
def MAE(y_true, y_pred):     
    
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    return np.mean(np.abs((y_true - y_pred)))

count = 0
now = datetime.now() 
name =str(now.year)+'_'+str(now.month)+'_'+str(now.day)+'_'+str(now.hour)

In [17]:
csv_test = pd.read_csv(src+'train.csv')
x = csv_test.iloc[:,4:-1].values
y = csv_test.iloc[:,0:4].values

X_train, X_test, Y_train, Y_test = train_test_split(x,y, test_size=0.001, random_state=1,shuffle=True)

In [18]:
sc = MinMaxScaler()

X_train_scale=sc.fit_transform(X_train)
Y_train_scale=sc.fit_transform(Y_train)
X_test_scale=sc.fit_transform(X_test)
Y_test_scale=sc.fit_transform(Y_test)

In [19]:
np.shape(X_train_scale)
X_train_scale_reshape = X_train_scale.reshape(809190, 225,1,1)
X_train_scale_reshape = X_test_scale.reshape(810, 225,1,1)
X_train_reshape = X_train.reshape(809190, 225,1,1)
X_test_reshape = X_test.reshape(810, 225,1,1)
print(np.shape(X_train_scale))

(809190, 225)


In [ ]:
def CNN(weights_path=None):

    model = Sequential()
    model.add(ZeroPadding2D((1,0),input_shape=(225,1,1)))
    model.add(Conv2D(600, 7, 1, activation='relu'))
    model.add(MaxPooling2D((7,1), strides=(1,2))) 
    model.add(Conv2D(300, 4, 1, activation='relu'))
    model.add(MaxPooling2D((4,1), strides=(1,2))) 
    model.add(Conv2D(100, 2, 1, activation='relu'))

    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(4, activation='linear'))

    return model

model=CNN()
model.summary()
#plot_model(model, to_file=save+str(count)+'_Architecture_'+name+'.png', show_shapes=True)

adam = keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, amsgrad=False)

model.compile(optimizer= adam , loss='mse', metrics=['mae','accuracy'])
early_stopping = EarlyStopping(patience=10)

hist = model.fit(X_train_reshape, Y_train_scale, batch_size=800, epochs=100,validation_split=0.01,callbacks=[early_stopping],verbose=1)

/home/lab1_ysy/anaconda3/envs/tf2.0/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(600, (7, 1), activation="relu")`
  """
/home/lab1_ysy/anaconda3/envs/tf2.0/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(300, (4, 1), activation="relu")`
  import sys
/home/lab1_ysy/anaconda3/envs/tf2.0/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(100, (2, 1), activation="relu")`
  if __name__ == '__main__':


Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_5 (ZeroPaddin (None, 227, 1, 1)         0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 221, 1, 600)       4800      
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 215, 1, 600)       0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 212, 1, 300)       720300    
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 209, 1, 300)       0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 208, 1, 100)       60100     
_________________________________________________________________
flatten_5 (Flatten)          (None, 20800)            

In [25]:
score = model.evaluate(X_test_reshape, Y_test_scale, verbose=0)
np.savetxt(save+str(count)+'_Score_'+name+'.txt', score)

print(model.metrics_names)
print(score)

fig, loss_ax = plt.subplots()
acc_ax = loss_ax.twinx()

loss_ax.plot(hist.history['loss'], label='train loss')
loss_ax.plot(hist.history['val_loss'], label='val_loss')
loss_ax.plot(hist.history['mean_absolute_error'], label='train acc')
loss_ax.plot(hist.history['val_mean_absolute_error'], label='val acc')
plt.savefig(save+str(count)+'_Train_curve_'+name+'.png', dpi=300)

In [ ]:
Y_hat_test = abs(model.predict(X_test_reshape,batch_size=32))
Y_hat_test_inverse = sc.inverse_transform(Y_hat_test)
Y_test_inverse = sc.inverse_transform(Y_test_scale)

record_pred = []


for i in range(100):

  Y_hat= list(map(int, Y_hat_test_inverse[i]))
  print(i,'real:',Y_test_inverse[i],'pred:',Y_hat,' ',sep='\n')
  Y_append= np.append(Y_test_inverse[i],Y_hat)
  record_pred.append(Y_append)  

np.savetxt(save+str(count)+'_Record pred_'+name+'.csv', record_pred,delimiter=',')
model.save_weights(save+str(count)+"_Weight_"+name+".h5")

final = MAE()
print('final mean_absolute_error:',)

In [ ]:
'''model_json = model.to_json()
with open("model.json", "w") as json_file : 
    json_file.write(model_json)
model.save_weights("model.h5")
print("Saved model to disk")

from keras.models import model_from_json 
json_file = open("model.json", "r") 
loaded_model_json = json_file.read() 
json_file.close() 
loaded_model = model_from_json(loaded_model_json)

loaded_model.load_weights("model.h5") 
print("Loaded model from disk")

loaded_model.compile(loss="mse", optimizer="adam", metrics=['mae'])

# model evaluation
score = loaded_model.evaluate(X_test_scale,Y_test_scale,verbose=0)

print("%s : %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))'''